In [ ]:
import sys
from pathlib import Path

from aiida import load_profile
from ase.io import read
from tb_hamiltonian.workflows import compute_bands

In [ ]:
sys.tracebacklimit = None

In [ ]:
_ = load_profile()

# Define parameters

In [ ]:
# Control parameters
debug = False
use_mpi = False


# Initial structure
structure_label = "BLG"
input_path = Path("examples/BLG/AB/rectangular")
initial_structure = read(input_path / "POSCAR", format="vasp")

# Distances
distances = [0.0, 1.425, 2.468, 2.850]

# Supercell
nx = 1  # number of repetitions in x direction
ny = 1  # number of repetitions in y direction
nz = 1  # number of repetitions in z direction

# Hamiltonian parameters
hopping_parameters = [0.0, -2.7, 0.0, -0.27]
interlayer_coupling = 0.33
nearest_neighbor = 1  # degree of nearest neighbors | don't use 0!

# Potential parameters
alpha = [1.0, 0.5]
onsite_term = 0.0
potential_type = "null"
potential_params = {
    "amplitude": 1.0,
    # "width": 0.5,
    # "height": 0.5,
}

# Band structure parameters
band_params = {
    "high_sym_points": {
        "Γ": (0.00000, 0.00000, 0.00000),
        "P": (0.00000, 0.33333, 0.00000),
        "X": (0.00000, 0.50000, 0.00000),
        "W": (0.50000, 0.50000, 0.00000),
        "Y": (0.50000, 0.00000, 0.00000),
    },
    "k_path": "Γ P X W Y Γ W",
    "points_per_segment": 100,
    "use_sparse_solver": False,
    "sparse_solver_params": {"k": 3, "sigma": 1e-8},
    "use_mpi": use_mpi,
    "fig_filename": "bands.png",
}

In [ ]:
computer = "localhost"

metadata = {
    "options": {
        # for more scheduler options, see https://aiida.readthedocs.io/projects/aiida-core/en/latest/topics/calculations/usage.html#options
        "prepend_text": "source $(conda info --base)/etc/profile.d/conda.sh && conda activate tbh",
    }
}

In [ ]:
wg = compute_bands(
    structure_label,
    initial_structure,
    [nx, ny, nz],
    distances,
    nearest_neighbor,
    hopping_parameters,
    interlayer_coupling,
    potential_type,
    potential_params,
    onsite_term,
    alpha,
    band_params,
    computer=computer,
    metadata=metadata,
)

In [ ]:
wg.submit()

In [ ]:
# ! verdi process status $wg.pk

In [ ]:
# from aiida.orm import load_node

# wg_node = load_node(wg.pk)
# while not wg_node.is_finished:
#     continue
# band_calculation = wg_node.called[-1]
# band_calculation.outputs.fig.value

In [ ]:
from aiida import orm

wg = orm.load_node(1086)